# 8puzzle Problem Solved using A* serach algorithm
A* search is simpely avoid expanding paths that are already expensive using an evaluation function "f(n)= g(n) + h(n)"
- **g(n)** = cost so far to reach n 
- **h(n)** = estimated cost from n to goal 
- **f(n)** = estimated total cost of path through nto goal


In [1]:
import collections #module that implements datatypes (list, dict,set)
import time

## Class Node
This class represent a solver node
* **puzzle**: is a puzzle instance ( or in other word a board)
* **parent**: is the preceding node generated by the solver
* **action**: is the action taken to produce puzzle

In [2]:
class Node:
    """
    A class representing an Solver node
    - 'puzzle' is a Puzzle instance
    - 'parent' is the preceding node generated by the solver
    - 'action' is the action taken to produce puzzle
    """
    def __init__(self, puzzle, parent=None, action=None):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        if (self.parent != None):
            self.g = parent.g + 1
        else:
            self.g = 0

    @property
    def state(self):
        """
        Return a hashable representation of self
        """
        return str(self)

    @property 
    def succ(self):
        """
        -Reconstruct a path from to the root 'parent'
        -Generates the node's children states.
        """
        node, p = self, []
        while node:
            p.append(node)
            node = node.parent
        yield from reversed(p) #yield is like return

    @property
    def solved(self):
        """ this function is to check if 'puzzle' is solved """
        return self.puzzle.solved

    @property
    def possible_moves(self):
        """is for 'actions' accessible at current state """
        return self.puzzle.possible_moves

    @property
    def h(self):
        """"h"""
        return self.puzzle.manhattan     # here we change the heuristic function(manhattan or misplaced)

    @property
    def f(self):
        """"f(n)=g(n) + h(n)"""
        return self.h + self.g

    """"convert this fun to string"""
    def __str__(self):
        return str(self.puzzle)



## Class Puzzle
This class represent an 8-puzzle with a board which is a square list (e.g  [ [ 1 ,2 ,3 ] , [ 4 ,0 ,6 ] , [7 ,5 ,8 ] ] )
* **__init__ :** constructor 
* **solved :** this function is to check if the puzzle is solved. The puzzle is solved if the flattened board's numbers are in increasing order from left to right and the '0' tile is in the last position on the board
* **move :** Return a new puzzle where 'at' and 'to' tiles have been swapped.
* **possible_moves :** Return a list of 'moves', 'actions' ( 'R','L','U','D' . 'move' can be called to return a new puzzle that results in sliding the '0' tile in the direction of 'action'
* **manhattan :** Is the heuristic function h(n) which is the distance from node n to the goal.
* **misplaced :** The second heuristic function. we get it by calculating number of misplaced tiles
* **copy: ** Return a new puzzle with the same board as 'self' (we clone the same puzzle)
* **puzzlePrint: ** Print the puzzle row by row
* **str: ** Same as print(self) except this returns a string
* **iter:** Make the class object iterative

In [3]:
class Puzzle:
    """
    A class representing an '8-puzzle'.
    - 'board' should be a square list of lists with integer entries 0...width^2 - 1
       e.g. [[1,2,3],[4,0,6],[7,5,8]]
    """
    def __init__(self, board):
        self.width = len(board[0])
        self.board = board

    @property
    def solved(self):
        """
        The puzzle is solved if the flattened board's numbers are in
        increasing order from left to right and the '0' tile is in the
        last position on the board
        """
        N = self.width * self.width
        """
        return true if str(self)= the string ['1','2','3','4','5','6','7','8','0']
        """
        flattenBoard= ''.join(map(str, range(1,N))) + '0'
        if str(self) == flattenBoard:
            return True
        else:
            return False

    
    
    def copy(self):
        """
        Return a new puzzle with the same board as 'self'
        """
        board = []
        for row in self.board:
            board.append([x for x in row])
        return Puzzle(board)
    
    def _move(self, at, to):
        """
        Return a new puzzle where 'at' and 'to' tiles have been swapped.
        NOTE: all moves should be 'actions' that have been executed
        """
        copy = self.copy()
        i, j = at
        r, c = to
        copy.board[i][j], copy.board[r][c] = copy.board[r][c], copy.board[i][j]
        return copy
    
    @property 
    def possible_moves(self):
        """
        Return a list of 'move', 'action' pairs. 'move' can be called
        to return a new puzzle that results in sliding the '0' tile in
        the direction of 'action'.
        """
        def create_move(at, to):
            return lambda: self._move(at, to)

        moves = []
        for i in range(self.width):
            for j in range(self.width):
                direcs = {'R':(i, j-1),'L':(i, j+1),'D':(i-1, j),'U':(i+1, j)}
                for action, (r, c) in direcs.items():
                    if r >= 0 and c >= 0 and r < self.width and c < self.width and self.board[r][c] == 0:
                        move = create_move((i,j), (r,c)), action
                        moves.append(move)
        return moves
    
    
    @property
    def manhattan(self):
        distance = 0
        for i in range(3):
            for j in range(3):
                if self.board[i][j] != 0:
                    Xstart= int((self.board[i][j]-1)/3)
                    Ystart= (self.board[i][j]-1)%3
                    distance += abs(Xstart - i) + abs(Ystart - j)
        return distance

    @property
    def misplaced(self):
        misplaced = 0
        count = 1
        for i in range(0,3):
            for j in range(0,3):
                if self.board[i][j]!= (count%9) and self.board[i][j] != 0:
                    misplaced += 1
                count+=1
        return misplaced
    
   

    """ Print the puzzle row by row """
    
    def puzzlePrint(self):
        for row in self.board:
            print(row)
        print("    |   ")
        print("    |   ")
        print("   \|/   ")

    
    
    """Same as print(self) except this returns a string"""
    def __str__(self):
        return ''.join(map(str, self))

    """ Make the class object iterative """
    def __iter__(self):
        for row in self.board:
            yield from row



## Class Solver
This class contains two A star search algorithms that solves the puzzle
1. **A* Graph Search Algorithm**
2. **A* Tree Search Algorithm**


In [4]:
class Solver:
    """
    An '8-puzzle' solver
    - 'start' is a Puzzle instance
    """
    def __init__(self, start):
        self.start = start

    
    
    def solveByGraph(self):
        """
        return a path to the solution, if it exists
        using A* tree algorithm
        """
        #deque is list-like container with fast appends and pops on either end
        queue = collections.deque([Node(self.start)]) #  the puzzle instance (we start with the root node)
        allNodes = set()                              # declare a set (fring)
        allNodes.add(queue[0].state)                  # put the initial board on that set
        
        while queue:                                  #while queue is not empty
            queue = collections.deque(sorted(list(queue), key=lambda node: node.f)) #we sorted a list of nodes ordered by key=f
            node = queue.popleft()                    # select from the fringe the left element (node with the lowest value)
            if node.solved:
                return node.succ # return all the successor of the selected node

            for move, action in node.possible_moves:
                child = Node(move(), node, action)

                if child.state not in allNodes:     # generates all the successors of the selected node
                    queue.appendleft(child)
                    allNodes.add(child.state)       
    
    
    def solveByTree(self):
        """
        return a path to the solution, if it exists
        using A* tree algorithm
        """
        queue = collections.deque([Node(self.start)]) #  the puzzle instance
        while queue:                                  #while queue is not empty
            queue = collections.deque(sorted(list(queue), key=lambda node: node.f)) #we sorted a list of nodes ordered by key=f
            node = queue.popleft()                    # select from the fringe the left element (node with the lowest value)
            if node.solved:
                return node.path # return all the successor of the selected node

            for move, action in node.possible_moves:
                child = Node(move(), node, action)
                queue.appendleft(child)



In [ ]:
def main():   
    board = [[1,0,4],[3,2,5],[6,7,8]]
   
     # Graph Search
    puzzle = Puzzle(board)
    s = Solver(puzzle)
    Start_time = int(round(time.time() * 1000))
    p = s.solveByGraph() # change between tree and graph search
    End_time = int(round(time.time() * 1000))

    steps = 0
    for node in p:
        print(node.action)
        node.puzzle.puzzlePrint()
        steps += 1
    print("********* Graph  Search*************")
    print("Total number of steps: " + str(steps))
    print("Total amount of time in search: " + str(End_time - Start_time) + " milisecond(ms)")

In [ ]:
main()